In [38]:
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.geography_utils import WGS84, CA_NAD83Albers_m

import pandas as pd
import geopandas as gpd

### `stop_key` for demo locations

diridon: `e170216debdf7de0d6f6b294bd5e6ee4`

In [23]:
identifiers = query_sql('''
select t1.feed_key, t1.gtfs_dataset_name, t2.analysis_name
from cal-itp-data-infra.mart_gtfs.fct_daily_schedule_feeds as t1
join cal-itp-data-infra.mart_transit_database.bridge_gtfs_analysis_name_x_ntd as t2
on t1.gtfs_dataset_name = t2.schedule_gtfs_dataset_name
where t1.date = "2026-02-11"

''')

# identifiers = query_sql('''
# select t1.feed_key, t1.gtfs_dataset_name
# from cal-itp-data-infra.mart_gtfs.fct_daily_schedule_feeds as t1
# where t1.date = "2026-02-11"

# ''')

In [14]:
identifiers

,feed_key,gtfs_dataset_name,analysis_name
0,2f6a644cede2fa1e2b862cb32ed694df,Bay Area 511 Marin Schedule,Marin County Transit District
1,05f168945848c5710debb7a264059d37,Pasadena Schedule,City of Pasadena
2,2e9b311ba31346e5f5941d6908482507,Monterey Salinas Schedule,Monterey-Salinas Transit
3,482c3440f66e22f2ba5141a6b49efd2f,Palos Verdes PTA Schedule,Palos Verdes Peninsula Transit Authority
4,01ecfa42d53a9c1a8a5dc65af767bfb9,Oregon POINT,POINT
...,...,...,...
211,e9ed5d14c378704e403f5c527cb0ce21,Bay Area 511 Tri-Valley Wheels Schedule,Livermore-Amador Valley Transit Authority
212,4ec8a98a0fcaab079b4bc8f5d0091ecf,Laguna Beach Flex,City of Laguna Beach
213,aba5c8b58430d025b241f7b20813e5df,Cudahy Schedule,City of Cudahy
214,3c87dab8c94c6205550a7ca9b31380e9,Bay Area 511 Golden Gate Ferry Schedule,"Golden Gate Bridge, Highway and Transportation..."


In [25]:
identifiers[identifiers.gtfs_dataset_name.str.contains('Santa Clara')]

,feed_key,gtfs_dataset_name,analysis_name
34,eef3ec6d4eda424edca5f8c6330aae15,Bay Area 511 Santa Clara Transit Schedule,Santa Clara Valley Transportation Authority


In [32]:
diridon_stops = query_sql('''
select service_date, stop_id, daily_arrivals, stop_name, pt_geom, feed_key, st_distance(t1.pt_geom, st_geogpoint(-121.90301, 37.32924)) as meters_from_key
from cal-itp-data-infra.mart_gtfs.fct_daily_scheduled_stops as t1
where service_date = "2026-02-11"
and st_distance(t1.pt_geom, st_geogpoint(-121.90301, 37.32924)) < 1000
'''
)

In [34]:
df = diridon_stops.merge(identifiers, on='feed_key')

In [35]:
type(df)

pandas.core.frame.DataFrame

In [40]:
gpd.GeoDataFrame(data=df, geometry=gpd.GeoSeries.from_wkt(df.pt_geom), crs=WGS84).drop(columns=['pt_geom'])

,service_date,stop_id,daily_arrivals,stop_name,feed_key,meters_from_key,gtfs_dataset_name,analysis_name,geometry
0,2026-02-11,868,4,W Santa Clara & Almaden Blvd,0cc4c66d9d61e84088a9e5008dcd4041,863.675914,Santa Cruz Schedule,Santa Cruz Metropolitan Transit District,POINT (-121.89520 37.33390)
1,2026-02-11,867,4,W Santa Clara & Delmas Ave,0cc4c66d9d61e84088a9e5008dcd4041,439.079762,Santa Cruz Schedule,Santa Cruz Metropolitan Transit District,POINT (-121.89942 37.33197)
2,2026-02-11,875,4,W Santa Clara & Almaden Blvd,0cc4c66d9d61e84088a9e5008dcd4041,818.219222,Santa Cruz Schedule,Santa Cruz Metropolitan Transit District,POINT (-121.89574 37.33379)
3,2026-02-11,871,4,W Santa Clara & Delmas Ave,0cc4c66d9d61e84088a9e5008dcd4041,445.319101,Santa Cruz Schedule,Santa Cruz Metropolitan Transit District,POINT (-121.89959 37.33218)
4,2026-02-11,865,29,Barack Obama Blvd & W San Carlos,0cc4c66d9d61e84088a9e5008dcd4041,606.581433,Santa Cruz Schedule,Santa Cruz Metropolitan Transit District,POINT (-121.90035 37.32421)
...,...,...,...,...,...,...,...,...,...
58,2026-02-11,60444,202,Santa Clara & Delmas,eef3ec6d4eda424edca5f8c6330aae15,505.659435,Bay Area 511 Santa Clara Transit Schedule,Santa Clara Valley Transportation Authority,POINT (-121.89886 37.33237)
59,2026-02-11,60442,202,Santa Clara & Almaden Ave,eef3ec6d4eda424edca5f8c6330aae15,975.732918,Bay Area 511 Santa Clara Transit Schedule,Santa Clara Valley Transportation Authority,POINT (-121.89421 37.33453)
60,2026-02-11,60409,204,Santa Clara & Delmas,eef3ec6d4eda424edca5f8c6330aae15,430.915869,Bay Area 511 Santa Clara Transit Schedule,Santa Clara Valley Transportation Authority,POINT (-121.89952 37.33194)
61,2026-02-11,60443,394,Santa Clara & Almaden Blvd,eef3ec6d4eda424edca5f8c6330aae15,814.300747,Bay Area 511 Santa Clara Transit Schedule,Santa Clara Valley Transportation Authority,POINT (-121.89575 37.33374)


## From collection to usable data

* need, essentially, a walking distance/time matrix (existing tools do 1 m/s straightline, we could too for now)
* then join to rt, figure out filters/precalc...